### This notebook is used for quick linear fitting

For a general case using features: $\{x_1, x_2, ... x_m\}$ to predict target y

1) y might depends on $x_i '$ to the power of p, so each feature $x_i$ will be expand to a set of new features including $x_i$ to the power of [$1 \over 2$, ${1}\over{3}$, $1\over 4$,1,2, 3, 4]

2) greedy choose features that predict target the best

3) add interation terms between chosen features and do greedy search again

3) fit the model with chosen features

In [44]:
import pandas as pd
import numpy as np
from numpy import meshgrid
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

In [2]:
# for testing purpose, use genXY to generate test sets
def genXY(d_row, d_feature):
    X = np.random.randn(d_row, d_feature)
    y = X.dot(np.random.randint(-5, 5, d_feature)) + 2*np.random.randn(d_row)
    return X, y

In [66]:
def loadDF(path):
    df = pd.read_csv(path)
    corr = df.corr()
    corr.style.background_gradient()
    #print(corr)
    display(corr)
    return df

In [67]:
df = loadDF(path)

,y,x1,x2
y,1.000000,0.262895,0.631638
x1,0.262895,1.000000,0.006580
x2,0.631638,0.006580,1.000000


In [4]:
path = "PredictiveModelingAssessmentData.csv"

In [32]:
df = pd.read_csv(path)

In [69]:
# correlation heat map
corr = df.corr()
corr.style.background_gradient().set_precision(1)

,y,x1,x2
y,1,0.3,0.6
x1,0.3,1,0.007
x2,0.6,0.007,1


In [70]:
# fn corrMap to quickly print corralation heat map for a df
def corrMap(df):
    corr = df.corr()
    corr.style.background_gradient()
    display(corr)

In [52]:
# expand current features to pows
def expandFeatures0(df, cols, pows):
    res_df = df.copy()
    for col in cols:
        for p in pows:
            new_col = str(p) + "_" + col
            res_df[new_col] = np.power(res_df[col], p)
    return res_df

def expandFeatures1(df, cols_pows):
    res_df = df.copy()
    for col, pows in cols_pows.items():
        for p in pows:
            new_col = str(np.round(p, 2)) + '_' + col
            res_df[new_col] = np.power(res_df[col], p)
    return res_df


In [53]:
cols = list(df.columns)
target = 'y'
cols.remove(target)
pows = [1/2, np.round(1/3, 2), 2, 4]
# make a dic for desired expansion for each features
cols_pows = {"x1": [2,3,4], "x2": [1/2,np.round(1/3, 2), 2, 3, 4]}
print("the columns to be expanded: ", cols)
print("the powers to be added as new features: ", pows)

the columns to be expanded:  ['x1', 'x2']
the powers to be added as new features:  [0.5, 0.33, 2, 4]


In [54]:
test_df = expandFeatures1(df, cols_pows)
test_df.head()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
0,1.300215,-0.054425,0.738897,0.002962,-0.000161,0.000009,0.859591,0.904967,0.545968,0.403414,0.298081
1,-0.805025,0.130174,0.977855,0.016945,0.002206,0.000287,0.988866,0.992637,0.956201,0.935026,0.914320
2,2.801926,1.749007,1.352562,3.059024,5.350253,9.357628,1.162997,1.104795,1.829423,2.474407,3.346788
3,3.123490,-0.979458,1.664484,0.959338,-0.939632,0.920330,1.290149,1.183102,2.770506,4.611463,7.675705
4,3.445728,0.300521,2.988848,0.090313,0.027141,0.008156,1.728829,1.435213,8.933214,26.700023,79.802321


In [42]:
# greedy features selection
def loss(model, X_test, y_test):
    pred = model.predict(X_test)
    return np.sum(np.power(pred-y_test, 2))/len(y_test)

def get_X_y(df, features, target):
    return np.array(df[features]), np.array(df[target])

def build_one(df_train, df_test, feature, target):
    reg_temp = LinearRegression(copy_X = True)
    X_train, y_train = get_X_y(df_train, feature, target)
    X_test, y_test = get_X_y(df_test, feature, target)
    reg_temp.fit(X_train, y_train)
    loss_temp = loss(reg_temp, X_test, y_test)
    return reg_temp, loss_temp, feature

def build_first(df_train, df_test, features, target):
    reg = LinearRegression(copy_X = True)
    cur_model, cur_loss, cur_feature = build_one(df_train, df_test, [features[0]], target)
    for f in features:
        new_model, new_loss, new_feature = build_one(df_train, df_test, [f], target)
        if new_loss < cur_loss:
            cur_model, cur_loss, cur_feature = new_model, new_loss, new_feature
    return cur_model, cur_loss, cur_feature

def build_new(cur_model, cur_loss, df_train, df_test, cur_features, new_features, target, flag):
    previous_loss = cur_loss
    
    for new_f in new_features:
        temp_features = cur_features + [new_f]
        reg_temp = LinearRegression(copy_X = True)
        X_train, y_train = get_X_y(df_train, temp_features, target)
        X_test, y_test = get_X_y(df_test, temp_features, target)
        reg_temp.fit(X_train, y_train)
        loss_temp = loss(reg_temp, X_test, y_test)
        if loss_temp < cur_loss:
            cur_model, cur_loss, cur_features = reg_temp, loss_temp, temp_features
    if cur_loss == previous_loss:
        flag = False
    return cur_model, cur_loss, cur_features, flag
        
def greedyForwards(df, features,target):
    # function loss(model, X_test, y_test) to compute the loss function of model 
    # first build new Model for each feature and select the best one -- build_one(features)
    # function build new Model build_new(cur_features, features_to_add, cur_model)
    # for each feature_to_add f, add f to cur_features, and build a ne multi linear regression model new_model, compare its loss with 
    # the current model by comparing loss(new_model, test_X, test_y) and loss(model, test_X, test_y) 
    # continue if the new model is better else stop and use the current model
    
    # first split the data into train and test sets
    df_copy = df.copy()
    df_train = df_copy.sample(frac = 0.75, random_state=404)
    #print("length,",len(df_train))
    df_test = df_copy.drop(df_train.index)
    # build first model with one feature
    model0, loss0, f0 = build_first(df_train, df_test, features, target)
    #build_new(cur_model, cur_loss, df_train, df_test, cur_features, new_features target, flag)
    flag = True
    model, loss, fs, flag = model0, loss0, f0, flag
    while flag:
        new_features = list(set(features)-set(fs))
        model, loss, fs, flag = build_new(model, loss, df_train, df_test, fs, new_features, target, flag)
    return model, fs, loss

In [55]:
test_fs = list(test_df.columns)
test_fs.remove(target)
print(test_fs, target)

['x1', 'x2', '2_x1', '3_x1', '4_x1', '0.5_x2', '0.33_x2', '2_x2', '3_x2', '4_x2'] y


In [56]:
test_df.head()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
0,1.300215,-0.054425,0.738897,0.002962,-0.000161,0.000009,0.859591,0.904967,0.545968,0.403414,0.298081
1,-0.805025,0.130174,0.977855,0.016945,0.002206,0.000287,0.988866,0.992637,0.956201,0.935026,0.914320
2,2.801926,1.749007,1.352562,3.059024,5.350253,9.357628,1.162997,1.104795,1.829423,2.474407,3.346788
3,3.123490,-0.979458,1.664484,0.959338,-0.939632,0.920330,1.290149,1.183102,2.770506,4.611463,7.675705
4,3.445728,0.300521,2.988848,0.090313,0.027141,0.008156,1.728829,1.435213,8.933214,26.700023,79.802321


In [59]:
model0, selected_features, loss0 = greedyForwards(test_df, test_fs, target)
print("selected_features are: ", selected_features, "the cost of current model is: ", np.round(loss0, 4))

selected_features are:  ['0.33_x2', 'x2', '2_x1', 'x1', '3_x2', '2_x2', '0.5_x2', '4_x1'] the cost of current model is:  1.253


In [72]:
#corrMap(test_df)
test_corr = test_df.corr()
test_corr.style.background_gradient()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
y,1,0.262895,0.631638,0.288778,0.2175,0.187632,0.690903,0.705617,0.492209,0.370317,0.281204
x1,0.262895,1,0.00658,-0.0143724,0.786745,-0.0239852,0.005115,0.00488736,0.0102168,0.0125753,0.0133092
x2,0.631638,0.00658,1,-0.00848253,0.00873011,-0.0199498,0.973844,0.950817,0.929242,0.787972,0.647022
2_x1,0.288778,-0.0143724,-0.00848253,1,-0.0384283,0.887963,-0.0078007,-0.00751554,-0.00850772,-0.00702759,-0.00562766
3_x1,0.2175,0.786745,0.00873011,-0.0384283,1,-0.077136,0.0107448,0.0117543,0.00725429,0.00721513,0.00726032
4_x1,0.187632,-0.0239852,-0.0199498,0.887963,-0.077136,1,-0.0209162,-0.0211151,-0.0175271,-0.0150875,-0.0130758
0.5_x2,0.690903,0.005115,0.973844,-0.0078007,0.0107448,-0.0209162,1,0.996126,0.830524,0.659352,0.514722
0.33_x2,0.705617,0.00488736,0.950817,-0.00751554,0.0117543,-0.0211151,0.996126,1,0.785933,0.609579,0.467565
2_x2,0.492209,0.0102168,0.929242,-0.00850772,0.00725429,-0.0175271,0.830524,0.785933,1,0.953339,0.857827
3_x2,0.370317,0.0125753,0.787972,-0.00702759,0.00721513,-0.0150875,0.659352,0.609579,0.953339,1,0.970374


In [76]:
# Now we have the primary selected features, interation terms are to be added. The strategy of choosing interactions is subjective
# Before a better method is found, get interactions for all combinations
# fn addInteraction:
def addInteractions(df, selected_features):
    res_df = df.copy()
    for i in range(len(selected_features)):
        for j in range(i+1, len(selected_features)):
            f1, f2 = selected_features[i], selected_features[j]
            new_col = 'cross_' + f1 + "_" + f2
            res_df[new_col] = res_df[f1] * res_df[f2]
    return res_df

In [78]:
cross_df = addInteractions(test_df, selected_features)
print(list(cross_df.columns))

['y', 'x1', 'x2', '2_x1', '3_x1', '4_x1', '0.5_x2', '0.33_x2', '2_x2', '3_x2', '4_x2', 'cross_0.33_x2_x2', 'cross_0.33_x2_2_x1', 'cross_0.33_x2_x1', 'cross_0.33_x2_3_x2', 'cross_0.33_x2_2_x2', 'cross_0.33_x2_0.5_x2', 'cross_0.33_x2_4_x1', 'cross_x2_2_x1', 'cross_x2_x1', 'cross_x2_3_x2', 'cross_x2_2_x2', 'cross_x2_0.5_x2', 'cross_x2_4_x1', 'cross_2_x1_x1', 'cross_2_x1_3_x2', 'cross_2_x1_2_x2', 'cross_2_x1_0.5_x2', 'cross_2_x1_4_x1', 'cross_x1_3_x2', 'cross_x1_2_x2', 'cross_x1_0.5_x2', 'cross_x1_4_x1', 'cross_3_x2_2_x2', 'cross_3_x2_0.5_x2', 'cross_3_x2_4_x1', 'cross_2_x2_0.5_x2', 'cross_2_x2_4_x1', 'cross_0.5_x2_4_x1']


In [80]:
cross_fs = list(cross_df.columns)
cross_fs.remove(target)
model1, cross_selected_features, loss1 = greedyForwards(cross_df, cross_fs, target)

In [81]:
print("selected_features are: ", cross_selected_features, "the cost of current model is: ", np.round(loss1, 4))

selected_features are:  ['0.33_x2', 'cross_2_x1_2_x2', 'x1', 'cross_0.33_x2_2_x2', 'cross_x2_3_x2', 'cross_2_x1_0.5_x2', 'cross_3_x2_2_x2', '0.5_x2', 'cross_x1_3_x2', 'cross_2_x1_4_x1', 'cross_0.33_x2_4_x1', 'cross_x2_2_x1', '2_x1', '4_x2', 'cross_0.33_x2_2_x1', 'cross_x1_2_x2', 'cross_x2_x1', 'cross_0.33_x2_x1'] the cost of current model is:  1.0111
